In [1]:
import numpy as np
from numba import njit, prange
from rtmag.test.eval import C_vec, C_cs, En_prime, Em_prime, eps

In [2]:
a1 = np.random.rand(512, 256, 256, 3).astype(np.float32)
a2 = np.random.rand(512, 256, 256, 3).astype(np.float32)
a2[:, :, 50] = 0

In [3]:
%%time
print(C_vec(a1, a2))
print(C_cs(a1, a2))
print(En_prime(a1, a2))
print(Em_prime(a1, a2))
print(eps(a1, a2))

0.7485102
0.797496182310815
0.3072035312652588
0.20669543696384807
1.0037494
CPU times: user 4.48 s, sys: 566 ms, total: 5.05 s
Wall time: 5.04 s


In [4]:
import numpy as np
from numba import prange, njit

@njit(parallel=True)
def dot_product(a, b):
    """
    a : [Nx, Ny, Nz, 3]
    b : [Nx, Ny, Nz, 3]
    """
    c = np.zeros(a.shape[:-1])
    I, J, K, L = a.shape
    for i in prange(I):
        for j in prange(J):
            for k in prange(K):
                for l in prange(L):
                    c[i, j, k] += a[i, j, k, l]*b[i, j, k, l]
    
    return c

def vector_norm(a):
    """
    a : [Nx, Ny, Nz, 3]
    """
    return np.sqrt(dot_product(a, a))


def C_vec(b, B):
    return dot_product(B, b).sum() / np.sqrt((vector_norm(B)**2).sum() * (vector_norm(b)**2).sum())


def C_cs(b, B):
    nu = dot_product(B, b)
    de = vector_norm(B) * vector_norm(b)
    M = np.sum([de!=0.])
    return (1 / M) * np.divide(nu, de, where=de!=0.).sum()


def En_prime(b, B):
    return 1 - (vector_norm(b - B).sum() / vector_norm(B).sum())


def Em_prime(b, B):
    nu = vector_norm(b - B)
    de = vector_norm(B)
    M = np.sum([de!=0.])
    return 1 - ((1 / M) * np.sum(np.divide(nu, de, where=de!=0.)))


def eps(b, B):
    return (vector_norm(b) ** 2).sum() / (vector_norm(B) ** 2).sum()

In [5]:
%%time
print(C_vec(a1, a2))
print(C_cs(a1, a2))
print(En_prime(a1, a2))
print(Em_prime(a1, a2))
print(eps(a1, a2))

0.7485081428288567
0.7974952024552032
0.3072030710673135
0.20669489899321525
1.0037486246532838
CPU times: user 4.9 s, sys: 1.7 s, total: 6.61 s
Wall time: 1.89 s


In [6]:
class Evaluator:

    def __init__(self, b, B):
        self.b = b
        self.B = B

    def compare(self):
        b = self.b
        B = self.B

        dot_product_b_B = dot_product(b, B)
        vector_norm_b = vector_norm(b)
        vector_norm_B = vector_norm(B)
        vector_norm_b_B = vector_norm(b-B)
        cs_de = vector_norm_b * vector_norm_B
        vector_norm_b_square_sum = (vector_norm(b)**2).sum()
        vecotr_norm_B_square_sum = (vector_norm(B)**2).sum()

        c_vec = dot_product_b_B.sum() / np.sqrt( vector_norm_b_square_sum * vecotr_norm_B_square_sum )
        c_cs  = (1 / np.sum([cs_de!=0.])) * np.divide(dot_product_b_B, cs_de, where=cs_de!=0.).sum()
        en_prime = 1 - (vector_norm_b_B.sum() / vector_norm_B.sum())
        em_prime = 1 - ((1 / np.sum([vector_norm_B!=0.])) * np.sum(np.divide(vector_norm_b_B, vector_norm_B, where=vector_norm_B!=0.)))
        eps = vector_norm_b_square_sum / vecotr_norm_B_square_sum

        print(c_vec)
        print(c_cs)
        print(en_prime)
        print(em_prime)
        print(eps)

In [7]:
evals = Evaluator(a1, a2)

In [8]:
%%time
evals.compare()

0.7485081428288567
0.7974952024552032
0.3072030710673135
0.20669489899321525
1.0037486246532838
CPU times: user 2.58 s, sys: 2.57 s, total: 5.14 s
Wall time: 959 ms


In [16]:
import numpy as np
from numba import prange, njit

@njit(parallel=True)
def dot_product(a, b):
    """
    a : [Nx, Ny, Nz, 3]
    b : [Nx, Ny, Nz, 3]
    """
    c = np.zeros(a.shape[:-1])
    I, J, K, L = a.shape
    for i in prange(I):
        for j in prange(J):
            for k in prange(K):
                for l in prange(L):
                    c[i, j, k] += a[i, j, k, l]*b[i, j, k, l]
    
    return c

@njit(parallel=True)
def vector_norm(a):
    """
    a : [Nx, Ny, Nz, 3]
    """
    return np.sqrt(dot_product(a, a))

# @njit(parallel=True)
def compare(b, B):
    dot_product_b_B = dot_product(b, B)
    vector_norm_b = vector_norm(b)
    vector_norm_B = vector_norm(B)
    nonzero_vector_norm_B = (vector_norm_B!=0.)
    vector_norm_b_B = vector_norm(b-B)
    cs_de = (vector_norm_b * vector_norm_B)
    nonzero_cs_de = (cs_de!=0.)
    vector_norm_b_square_sum = (vector_norm(b)**2).sum()
    vecotr_norm_B_square_sum = (vector_norm(B)**2).sum()

    c_vec = dot_product_b_B.sum() / np.sqrt( vector_norm_b_square_sum * vecotr_norm_B_square_sum )
    c_cs  = (1 / nonzero_cs_de.sum()) * np.sum(dot_product_b_B[nonzero_cs_de]cs_de[nonzero_cs_de])
    en_prime = 1 - (vector_norm_b_B.sum() / vector_norm_B.sum())
    em_prime = 1 - ((1 / nonzero_vector_norm_B.sum()) * np.sum(vector_norm_b_B[nonzero_vector_norm_B], vector_norm_B[nonzero_vector_norm_B]))
    eps = vector_norm_b_square_sum / vecotr_norm_B_square_sum

    print(c_vec)
    print(c_cs)
    print(en_prime)
    print(em_prime)
    print(eps)

In [17]:
compare(a1, a2)

MemoryError: Allocation failed (probably too large).

In [50]:
%%time
compare(a1, a2)

MemoryError: Allocation failed (probably too large).